# Vortex Solver with linearized gravity

The aim og this notebook is to haVe an expliit function of $T_{\mu,\nu}$ to build the EOM of linearized graVity $h_{\mu,\nu}$

In [1]:
%pylab inline
import cupy as cp
import h5py
import copy

Populating the interactive namespace from numpy and matplotlib


### Equations

In [2]:
### Padded
def Dx(u,delta):
  d = cp.zeros_like(u)
  d[1:-1,:,:] += (u[2:,:,:] - u[:-2,:,:])/(2*delta)
  d[0,:,:] += (u[1,:,:] - u[0,:,:])/delta
  d[-1,:,:] += (u[-1,:,:] - u[-2,:,:])/delta
  return d

def Dy(u,delta):
  d = cp.zeros_like(u)
  d[:,1:-1,:] += (u[:,2:,:] - u[:,:-2,:])/(2*delta)
  d[:,0,:] += (u[:,1,:] - u[:,0,:])/delta
  d[:,-1,:] += (u[:,-1,:] - u[:,-2,:])/delta
  return d

def Dz(u,delta):
  d = cp.zeros_like(u)
  d[:,:,1:-1] += (u[:,:,2:] - u[:,:,:-2])/(2*delta)
  d[:,:,0] += (u[:,:,1] - u[:,:,0])/delta
  d[:,:,-1] += (u[:,:,-1] - u[:,:,-2])/delta
  return d


def Lap(u,delta):
  d = cp.zeros_like(u)
  d[1:-1,:,:] += (u[2:,:,:] -2*u[1:-1,:,:] + u[:-2,:,:])
  d[0,:,:]    += u[1,:,:] -2*u[0,:,:] + u[0,:,:]#u[2,:,:] -2*u[1,:,:] + u[0,:,:]
  d[-1,:,:]   += u[-1,:,:] -2*u[-1,:,:] + u[-2,:,:]#u[-3,:,:] -2*u[-2,:,:] + u[-1,:,:]

  d[:,1:-1,:] += (u[:,2:,:] -2*u[:,1:-1,:] + u[:,:-2,:])
  d[:,0,:]    += u[:,1,:] -2*u[:,0,:] + u[:,0,:]#u[:,2,:] -2*u[:,1,:] + u[:,0,:]
  d[:,-1,:]   += u[:,-1,:] -2*u[:,-1,:] + u[:,-2,:]#u[:,-3,:] -2*u[:,-2,:] + u[:,-1,:]
    
  d[:,:,1:-1] += (u[:,:,2:] -2*u[:,:,1:-1] + u[:,:,:-2])
  d[:,:,0]    += u[:,:,1] -2*u[:,:,0] + u[:,:,0]#u[:,:,2] -2*u[:,:,1] + u[:,:,0]
  d[:,:,-1]   += u[:,:,-1] -2*u[:,:,-1] + u[:,:,-2]#u[:,:,-3] -2*u[:,:,-2] + u[:,:,-1]

  d[:,:,:] *= 1/delta**2

  return d

In [3]:
def EOM(fields,delta,lam,Q,V,omega,G):
    
    T = cp.tile(cp.zeros_like(fields[0]),(20,1))
    T00 = T[0]
    T01 = T[1]
    T02 = T[2]
    T03 = T[3]
    T11 = T[4]
    T12 = T[5]
    T13 = T[6]
    T22 = T[7]
    T23 = T[8]
    T33 = T[9]
    
    piT00 = T[10]
    piT01 = T[11]
    piT02 = T[12]
    piT03 = T[13]
    piT11 = T[14]
    piT12 = T[15]
    piT13 = T[16]
    piT22 = T[17]
    piT23 = T[18]
    piT33 = T[19]


    ### matter
    #################
    
    phi_r= fields[0]
    phi_i= fields[1]
    chi1 = fields[2]
    chi2 = fields[3]
    chi3 = fields[4]
    Ax   = fields[5]
    Ay   = fields[6]
    Az   = fields[7]
    At   = fields[8]
    
    ### gravity
    #################
    h00  = fields[9]
    h01  = fields[10]
    h11  = fields[11]
    h22  = fields[12]
    h33  = fields[13]
    h12  = fields[14]
    h13  = fields[15]
    h23  = fields[16]
    h02  = fields[17]
    h03  = fields[18]
    
     
    ### moments fields
    #################
    dt_phir = fields[19]
    dt_phii = fields[20]
    dt_chi1 = fields[21]
    dt_chi2 = fields[22]
    dt_chi3 = fields[23]
    dt_Ax   = fields[24]
    dt_Ay   = fields[25]
    dt_Az   = fields[26]
    dt_At   = fields[27]

    
    ### moments gravity
    #################
    dt_h00  = fields[28]
    dt_h01  = fields[29]
    dt_h22  = fields[30]
    dt_h33  = fields[31]
    dt_h11  = fields[32]
    dt_h12  = fields[33]
    dt_h13  = fields[34]
    dt_h23  = fields[35]
    dt_h02  = fields[36]
    dt_h03  = fields[37]

    
    ### Spatial derivatives
    #################
    dx_phir  = Dx(fields[0],delta)
    dy_phir  = Dy(fields[0],delta)
    dz_phir  = Dz(fields[0],delta)
    
    dx_phii  = Dx(fields[1],delta)
    dy_phii  = Dy(fields[1],delta)
    dz_phii  = Dz(fields[1],delta)
    
    dx_chi1 =  Dx(fields[2],delta)
    dy_chi1 =  Dy(fields[2],delta)
    dz_chi1 =  Dz(fields[2],delta)
    
    dx_chi2 =  Dx(fields[3],delta)
    dy_chi2 =  Dy(fields[3],delta)
    dz_chi2 =  Dz(fields[3],delta)
    
    dx_chi3 =  Dx(fields[4],delta)
    dy_chi3 =  Dy(fields[4],delta)
    dz_chi3 =  Dz(fields[4],delta)
    
    dx_Ax   =  Dx(fields[5],delta)
    dy_Ax   =  Dy(fields[5],delta)
    dz_Ax   =  Dz(fields[5],delta)
    
    dx_Ay   =  Dx(fields[6],delta)
    dy_Ay   =  Dy(fields[6],delta)
    dz_Ay   =  Dz(fields[6],delta)
    
    dx_Az   =  Dx(fields[7],delta)
    dy_Az   =  Dy(fields[7],delta)
    dz_Az   =  Dz(fields[7],delta)
    
    dx_At   =  Dx(fields[8],delta)
    dy_At   =  Dy(fields[8],delta)
    dz_At   =  Dz(fields[8],delta)
    
    ###### Common Terms ######
    #################
    d_phis2   = dx_phii**2 + dx_phir**2 + dy_phii**2 + dy_phir**2 + dz_phii**2 + dz_phir**2
    modA2     = -At**2 + Ax**2 + Ay**2 + Az**2
    mod_phi   = phi_r**2 + phi_i**2
    mod_chi   = chi1**2 + chi2**2 + chi3**2
    dtAx_dxAt = (dt_Ax - dx_At)
    dtAy_dyAt = (dt_Ay - dy_At)
    dtAz_dzAt = (dt_Az - dz_At)
    dxAy_dyAx = (dx_Ay - dy_Ax)
    dxAz_dzAx = (dx_Az - dz_Ax)
    dyAz_dzAy = (dy_Az - dz_Ay)
    DtphirAphii = dt_phir + At* Q* phi_i
    DtphiiAphir = dt_phii - At* Q* phi_r
    DxphirAphii = dx_phir + Ax* Q* phi_i
    DxphiiAphir = dx_phii - Ax* Q* phi_r
    DyphirAphii = dy_phir + Ay* Q* phi_i
    DyphiiAphir = dy_phii - Ay* Q* phi_r    
    DzphirAphii = dz_phir + Az* Q* phi_i
    DzphiiAphir = dz_phii - Az* Q* phi_r
    
    ######## Scalars ########
    #################
    ### DphiDphic ###
    Tensor4 = -(  -(dt_phii**2 + dt_phir**2) + d_phis2 +  modA2 * Q**2* mod_phi+\
                  2*Q *((-At* dt_phir + Ax* dx_phir + Ay* dy_phir +Az *dz_phir)* phi_i  +  (At *dt_phii-Ax* dx_phii - Ay* dy_phii - Az *dz_phii)*phi_r) )
    ###### F2 #######
    Tensor4 +=1/4*( -2* (dtAx_dxAt**2 + dtAy_dyAt**2 - dxAy_dyAx**2 + dtAz_dzAt**2 -dxAz_dzAx**2 - dyAz_dzAy**2) )
    
    #### dChi2 ######
    Tensor4 +=-( -dt_chi1**2 - dt_chi2**2 - dt_chi3**2 + dx_chi1**2 + dx_chi2**2 + dx_chi3**2 + dy_chi1**2 + dy_chi2**2 + dy_chi3**2 + dz_chi1**2 + dz_chi2**2 + dz_chi3**2 )
    
    #### Potential###
    Tensor4 += lam * (mod_chi - V**2)**2 + mod_chi * omega**2 +  lam * mod_phi * (2*(mod_chi - V**2) + mod_phi)
    
    
    #########################
    #### EQS gravity ########
    #########################
    ### Tensor 1 + Tensor 2 + Tensor 3 + gmunu scalars ##
    
    ##_00_##
    T00 = 2* (DtphirAphii**2 + DtphiiAphir**2)                                
    T00 += dtAx_dxAt**2 +dxAy_dyAx**2 + dxAz_dzAx**2                          
    T00 += 2* (dt_chi1**2 + dt_chi2**2 + dt_chi3**2)                          
    T00 -= Tensor4
    
    
    ##_01_##
    T01 = -2*(DtphirAphii * DxphirAphii + DtphiiAphir *DxphiiAphir)
    T01 += dtAy_dyAt * dxAy_dyAx + dtAz_dzAt * dxAz_dzAx
    T01 +=  2 * (dt_chi1 * dx_chi1 + dt_chi2 * dx_chi2 + dt_chi3 * dx_chi3)
    
    
    ##_02_##
    T02 = -2*(DtphirAphii * DyphirAphii + DtphiiAphir * DyphiiAphir)
    T02 += - dtAx_dxAt * dxAy_dyAx + dtAz_dzAt * dyAz_dzAy
    T02 += 2*(dt_chi1 * dy_chi1 + dt_chi2* dy_chi2 + dt_chi3 *dy_chi3)
    
    
    ##_03_##
    T03 = -2*(DtphirAphii * DzphirAphii + DtphiiAphir * DzphiiAphir)
    T03 += -(dtAx_dxAt *dxAz_dzAx + dtAy_dyAt *dyAz_dzAy)
    T03 += 2 *(dt_chi1 *dz_chi1 + dt_chi2* dz_chi2 + dt_chi3* dz_chi3)
    
  
    ##_11_##
    T11 = 2*(DxphirAphii**2 + DxphiiAphir**2)
    T11 += -dtAx_dxAt**2 + dxAy_dyAx**2 + dxAz_dzAx**2
    T11 += 2 *(dx_chi1**2 + dx_chi2**2 + dx_chi3**2)
    T11 += Tensor4
    
    
    ##_12_##
    T12 = 2*(DxphirAphii * DyphirAphii + DxphirAphii * DyphiiAphir)
    T12 += - dtAx_dxAt*dtAy_dyAt  + dxAz_dzAx * dyAz_dzAy
    T12 += 2* (dx_chi1* dy_chi1 + dx_chi2* dy_chi2 + dx_chi3* dy_chi3)

    
    ##_13_##
    T13 = 2*(DxphirAphii*DzphirAphii + DxphiiAphir*DzphiiAphir)
    T13 += -dtAx_dxAt * dtAz_dzAt  - dxAy_dyAx*dyAz_dzAy
    T13 += 2*(dx_chi1* dz_chi1 + dx_chi2* dz_chi2 + dx_chi3* dz_chi3)
    
    
    ##_22_##
    T22 = 2*(DyphirAphii**2 + DyphiiAphir**2)
    T22 += - dtAy_dyAt**2  + dxAy_dyAx**2 + dyAz_dzAy**2
    T22 += 2 *(dy_chi1**2 + dy_chi2**2 + dy_chi3**2)
    T22 += Tensor4
    
    
    ##_23_##
    T23 = 2*(DyphirAphii * DzphirAphii + DyphiiAphir * DzphiiAphir)
    T23 += -dtAy_dyAt*dtAz_dzAt + dxAy_dyAx*dxAz_dzAx
    T23 += 2 *(dy_chi1* dz_chi1 + dy_chi2* dz_chi2 + dy_chi3* dz_chi3)
                       
    
    ##_33_##
    T33 = 2*(DzphirAphii**2  +DzphiiAphir**2)
    T33 += -dtAz_dzAt**2  +dxAz_dzAx**2 + dyAz_dzAy**2
    T33 += 2 *(dz_chi1**2 + dz_chi2**2 + dz_chi3**2)
    T33 += Tensor4


    T_trace = T00 - T11 - T22 - T33
    
    T00 -= 1/2 *T_trace
    T11 += 1/2 *T_trace
    T22 += 1/2 *T_trace
    T33 += 1/2 *T_trace
    
    
    #### EQS hs
    #################
    eqh00 = Lap(h00,delta) -  1e-5*T00 #16*cp.pi*G* T00
    eqh01 = Lap(h01,delta) -  1e-5*T01#16*cp.pi*G* T01
    eqh02 = Lap(h02,delta) -  1e-5*T02#16*cp.pi*G* T02
    eqh03 = Lap(h03,delta) -  1e-5*T03#16*cp.pi*G* T03
    eqh11 = Lap(h11,delta) -  1e-5*T11#16*cp.pi*G* T11
    eqh12 = Lap(h12,delta) -  1e-5*T12#16*cp.pi*G* T12
    eqh13 = Lap(h13,delta) -  1e-5*T13#16*cp.pi*G* T13
    eqh22 = Lap(h22,delta) -  1e-5*T22#16*cp.pi*G* T22
    eqh23 = Lap(h23,delta) -  1e-5*T23#16*cp.pi*G* T23
    eqh33 = Lap(h33,delta) -  1e-5*T33#16*cp.pi*G* T33
    ##############################################################################################
    
    #########################
    #### EQS MATTER ########
    #########################

    mod_phi = 2*lam*(phi_r**2 + phi_i**2)
    mod_chi = 2*lam*(chi1**2 +chi2**2 +chi3**2 -V**2)
    mod_A   = Q**2*(Ax**2 + Ay**2 + Az**2 - At**2)    
    
    mod_chi += mod_phi
    mod_A   += mod_chi
    
    
    #####################
    ####### higgs #######
    
    ## aux calcs
    
    AmuDmuphi_r = Ax*dx_phir
    AmuDmuphi_r += Ay*dy_phir
    AmuDmuphi_r += Az*dz_phir
    AmuDmuphi_r -= At*dt_phir#fields[9]
    AmuDmuphi_r *= 2*Q
    
    AmuDmuphi_i = Ax*dx_phii
    AmuDmuphi_i += Ay*dy_phii
    AmuDmuphi_i += Az*dz_phii
    AmuDmuphi_i -= At*dt_phii#fields[10]
    AmuDmuphi_i *= 2*Q
    
    ## Eqs
    
    lap = Lap(phi_r,delta)
    eq1 = lap - mod_A*phi_r - AmuDmuphi_i
    
    lap[:,:,:] = Lap(phi_i,delta)
    eq2 = lap - mod_A*phi_i + AmuDmuphi_r

    #####################
    ####### chis ########
    
    mod_chi += omega**2
    
    lap[:,:,:] = Lap(chi1,delta)
    eq3     = lap - mod_chi*chi1
    
    lap[:,:,:] = Lap(chi2,delta)
    eq4     = lap - mod_chi*chi2
    
    lap[:,:,:]  = Lap(chi3,delta)
    eq5     = lap -mod_chi*chi3

    
    #####################
    ####### As ##########
    
    ### aux calc
    mod_phi *= 1/lam * Q**2
    
    dx_phir *= phi_i
    dx_phir -= dx_phii*phi_r
    dx_phir *= 2*Q
    
    dy_phir *= phi_i
    dy_phir -= dy_phii*phi_r
    dy_phir *= 2*Q

    dz_phir *= phi_i
    dz_phir -= dz_phii*phi_r
    dz_phir *= 2*Q

    #### eqs
    lap[:,:,:] = Lap(Ax,delta)
    eq6 = lap - Ax*mod_phi + dx_phir
    
    lap[:,:,:] = Lap(Ay,delta)
    eq7 = lap - Ay*mod_phi + dy_phir
    
    lap[:,:,:] = Lap(Az,delta)
    eq8 = lap - Az*mod_phi + dz_phir
    
    lap[:,:,:] = Lap(At,delta)
    eq9 = lap - At*mod_phi - 2*Q*(phi_r*dt_phii - phi_i*dt_phir)#fields[9] )

        
    return cp.array([dt_phir,dt_phii,dt_chi1,dt_chi2,dt_chi3,dt_Ax,dt_Ay,dt_Az,dt_At,\
                    dt_h00,dt_h01,dt_h11,dt_h22,dt_h33,dt_h12,dt_h13,dt_h23,dt_h02,dt_h03,\
                     eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8,eq9,\
                     eqh00,eqh01,eqh22,eqh33,eqh11,eqh12,eqh13,eqh23,eqh02,eqh03])


## Rk4

In [4]:
def OneStep(fields,dt,k):
  out = fields + k*dt
  return out 

def RK4step(fields,delta,dt,lam,Q,V,omega,G):
  #k = EqDif(fields,delta,lam,Q,V,omega) ## k1
  k = EOM(fields,delta,lam,Q,V,omega,G) ## k1

  f_aux = OneStep(fields,dt/2,k) ## u1 = u +dt/2*k1

  #k_aux = EqDif(f_aux,delta,lam,Q,V,omega) ## k2
  k_aux = EOM(f_aux,delta,lam,Q,V,omega,G) ## k2
  k[:,:,:,:] += 2*k_aux ## armo el ultimo k

  f_aux[:,:,:,:] = OneStep(fields,dt/2,k_aux) ## u2
  #k_aux[:,:,:,:] = EqDif(f_aux,delta,lam,Q,V,omega) ## k3
  k_aux[:,:,:,:] = EOM(f_aux,delta,lam,Q,V,omega,G) ## k3

  k[:,:,:,:] += 2*k_aux ## armo el ultimo k
  f_aux[:,:,:,:] = OneStep(fields,dt,k_aux) ## u3

  #k_aux[:,:,:,:] = EqDif(f_aux,delta,lam,Q,V,omega) ## k4 
  k_aux[:,:,:,:] = EOM(f_aux,delta,lam,Q,V,omega,G) ## k4 
  k[:,:,:,:] += k_aux 
  f_aux[:,:,:,:] = OneStep(fields,dt/6,k) ## new u

  return f_aux

# SIMULATION

### Pars

In [5]:
L0 = 65.2552
Nx = 101
delta = L0/(Nx-1)
### PARS0
delta0 = delta #0.333024
dt0 = 0.001
Q0 = 0.212132
lam0 = 0.25
V0 = 5.
omega0 = 0.25
N0 = 60000
G = 1
pars0 = [N0,delta0,dt0,lam0,Q0,V0,omega0,G]



hkeys = ['checkpointInterVal',
 'dt',
 'kerOrderX',
 'kerOrderY',
 'kerOrderZ',
 'lR',
 'lX',
 'lY',
 'lZ',
 'nR',
 'nX',
 'nY',
 'nZ',
 'q',
 'rA',
 'rχ',
 'rϕ',
 'v',
 'λ',
 'χ0',
 'ω',
 'ϵ']
hValues0 = [500,dt0,2,2,2,sqrt(2)*100,L0,L0,L0,6001,Nx,Nx,Nx,Q0,1.34241,2.33117,0.799732,V0,lam0,4.79612,omega0,10e-5]

snap_list = np.arange(0,N0+1,500)


In [6]:
def name2process(path,name):
    return rename(path+name,path+name[:-3] +'_processing.h5' )
def name2done(path,name):
    return rename(path+name[:-3] +'_processing.h5',path+name[:-2] +'_done.h5' )


In [7]:
def leadSeed(path,name):
    with h5py.File(path+name, 'r') as f:
        # List all groups
        #print("Keys: %s" % f.keys())
        a_group_key = list(f.keys())[0]
        #print(a_group_key)
        # Get the data
        with cp.cuda.Device(0):
            data0_ = cp.array(f[a_group_key])
            data0  = cp.empty((38,len(data0_[0,0,:]),len(data0_[0,0,:]),len(data0_[0,0,:])))
            
            dth02  = cp.zeros_like(data0_[0])
            dth03  = cp.zeros_like(data0_[0])
            h02    = cp.zeros_like(data0_[0])
            h03    = cp.zeros_like(data0_[0])
            
            for i in range(17):
                data0[i] = data0_[i]
            data0[17] = h02[17]
            data0[18] = h03[18]
            for i in range(17):
                data0[i+19] = data0_[i+17]                 
            data0[36] = dth02[36]
            data0[37] = dth03[37]
                        
    return data0
        

In [8]:
def SimulationRun(all_fields0,pars0,hkeys,hValues0,simul_path,name):
    N0,delta0,dt0,lam0,Q0,V0,omega0,G0 = pars0

    N = N0
    ## sim GPU0
    hf0 = h5py.File(simul_path+name[4:]+"test", 'w')
    for k,Val in zip(hkeys,hValues0):
        hf0.attrs[k] = Val

    i = 0
    while i <= N:
        with cp.cuda.Device(0):
            if i in snap_list:
            ### GUARDO FIELDS 0
                fields = cp.asnumpy(all_fields0)
                string_name = '0'*(10-len(str(i))) + str(i)
                grp = hf0.create_group(string_name)
                Dus = grp.create_dataset("Du", data = fields[19:])
                us = grp.create_dataset("u", data = fields[:19])
               
            all_fields0[:,:,:,:] = RK4step(all_fields0,delta0,dt0,lam0,Q0,V0,omega0,G0)
            
        i +=1
        print(i)
    hf0.close()

    return



In [9]:
from os import listdir
from os.path import isfile, join
from os import rename
import time


mypath = 'G:\\gravity\\seeds\\'
simul_path = 'G:\\gravity\\simulation\\'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
file = onlyfiles[-1]

['seed101_v6lX65alpha90beta0._done.h5', 'seed101_v6lX65alpha90beta180_done.h5', 'seed101_v6lX65alphazerobetazero_done.h5', 'seed201_v6lX65alphazerobetazero.h5']


In [10]:
all_fields = leadSeed(mypath,file)
#print("Im runnin'", file)
#SimulationRun(all_fields,pars0,hkeys,hValues0,simul_path,file)

In [11]:
SimulationRun(all_fields,pars0,hkeys,hValues0,simul_path,file)

OutOfMemoryError: Out of memory allocating 2,468,662,784 bytes (allocated so far: 16,695,965,184 bytes).

time101 = 20 segs per 100, 3hs 20 min for 60k
